In [17]:
import torch
import torch.nn as nn
import pandas as pd
from transformers import AutoTokenizer

In [18]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

sentence = "The bank approved the loan"
tokens = tokenizer.tokenize(sentence)

print("Subword Tokens:")
print(tokens)

Subword Tokens:
['the', 'bank', 'approved', 'the', 'loan']


In [19]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
token_ids = torch.tensor([token_ids])  # batch size = 1

print("Token IDs:")
print(token_ids)

Token IDs:
tensor([[1996, 2924, 4844, 1996, 5414]])


In [20]:
vocab_size = tokenizer.vocab_size
embedding_dim = 8

embedding_layer = nn.Embedding(vocab_size, embedding_dim)

embeddings = embedding_layer(token_ids)

print("Embeddings shape:", embeddings.shape)

Embeddings shape: torch.Size([1, 5, 8])


In [21]:
multihead_attn = nn.MultiheadAttention(
    embed_dim=8,
    num_heads=2,
    batch_first=True
)

In [22]:
output, attention_weights = multihead_attn(
    embeddings,  # Query
    embeddings,  # Key
    embeddings   # Value
)

In [23]:
attention_df = pd.DataFrame(
    attention_weights[0].detach().numpy(),
    index=tokens,
    columns=tokens
)

attention_df

,the,bank,approved,the,loan
the,0.193887,0.180280,0.213396,0.193887,0.218550
bank,0.206953,0.183605,0.247156,0.206953,0.155332
approved,0.190035,0.384522,0.104589,0.190035,0.130819
the,0.193887,0.180280,0.213396,0.193887,0.218550
loan,0.167137,0.195587,0.199707,0.167137,0.270431


## How to Read This Table (Sentence-Based)

- **Rows** → the **current token**
- **Columns** → the tokens it **attends to**

## Example Interpretation

If the row **`bank`** has higher values under:

- `approved`
- `loan`

### What This Means
- To understand **“bank”**, the model focuses more on **approved** and **loan**.
- From this context, the model learns:

> **bank = financial institution**

Not a river bank.


## What Multi-Head Attention Is Doing Here

Because **2 attention heads** are used:

### Head 1
- Focuses on **semantic meaning**
- Example:

bank ↔ loan


### Head 2
- Focuses on **action or relationship**
- Example:

approved ↔ bank


### Final Idea
- Each attention head looks at the sentence **from a different perspective**.
- All heads run **in parallel**.
- Their outputs are **combined** to form a richer, context-aware representation.


In [24]:
print("Final output shape:", output.shape)

Final output shape: torch.Size([1, 5, 8])
